In [1]:
import numpy as np

# Instructions de flot de contrôle:
# - Sequence: retour à la ligne
# - Branchement: if: // elif: // else: //
# - No-op: pass
# - Boucle for: for i in range(start, stop, step): //
# - Boucle while: while cond: //
# - Sortie boucle: break
# - Itération suivante: continue


# Liste des valeurs définies en mini-python
from numpy import Infinity

# Liste des fonctions définies en mini-python
min, max
from numpy import sqrt
from numpy import floor
from numpy import abs
from numpy import exp, log
from numpy import sin, cos, tan
from numpy import arcsin, arccos, arctan
from numpy import sinh, cosh, tanh
from numpy import arcsinh, arccosh, arctanh
from numpy import mod

def size(t):
    shape = np.shape(t)
    if np.ndim(t) == 1:
        return shape[0]
    else:
        return shape

def rand():
    return np.random.random()

def vector(n):
    if np.isscalar(n):
        return np.zeros(n)
    else:
        return np.array(n)
    
def matrix(n, m):
    return np.zeros((n, m))

def tensor(*args):
    return np.zeros(args)

def array(a):
    return np.array(a)

def length(s):
    return len(s)

def substr(s, begin, length):
    return s[begin:begin+length]

$\DeclareMathOperator{\moy}{moy}$

# Introduction

**Problème**
- Un algorithme *est-il efficace* ?
- Sur quels critères mesure-ton l'efficacité d'un algorithme ?
  + Rapidité de calcul ?
  + Espace mémoire requis pour le faire tourner ?
  + Cas le plus défavorable ? en moyenne ?

**Solution**:
- Ne pas quantifier précisément le temps de calcul et l'espace mémoire requis
- Classifier les algorithmes dans différents **ordres de grandeurs asymptotiques** !

# Exemple d'introduction

Suite de Fibonnaci $(f_n)_{n\in\mathbb{N}}$ définie par récurrence par :
$$
\begin{align}
f_0 = f_1 &= 1 \\
\forall n \in \mathbb{N}. f_{n+2} &= f_{n+1} + f_{n}
\end{align}
$$

## Implémentation avec fonction récursive :

In [2]:
def fibo(n):
    if n < 2:
        return 1
    return fibo(n - 1) + fibo(n - 2)

fibo(6)

13

**Arbre d'appel de la fonction `fibo`** :

![Arbre appel fibo](./assets/fibo.svg)

**Opérations élementaires** :
  - Additions
  
Nombre d'étages complets de l'arbre est $\lfloor n/2 \rfloor + 1$

En somme au rang `n`, cela nécessite un nombre d'additions de l'ordre de :
$$
\begin{align}
T(n) &= \sum^{\lfloor n/2 \rfloor}_{i=0} 2^{i} \\
&= 2^{\lfloor n/2 \rfloor + 1} - 1 \\
&\approx 2 \times 1.4414^{n}
\end{align}
$$

- Algorithme redondant (on recalcule plusieurs fois le même rang)
- Le nombre d'opérations à effectuer est multiplié à chaque unité ajouté à `n`
- Calcul de `fibo(100)` nécessite $\approx 2 * 10^{16}$ opérations.

## Implémentation avec calcul matriciel simple

$$
\begin{align}
\begin{pmatrix}
f_{n+2} \\
f_{n+1}
\end{pmatrix}
&= A
\begin{pmatrix}
f_{n+1} \\
f_{n}
\end{pmatrix}
&
\text{ avec : }
&
A =
\begin{pmatrix}
1 1 \\
f_{n}
\end{pmatrix}
\end{align}
$$

En fonction des premiers termes de la suite:
$$
\begin{pmatrix}
f_{n+1} \\
f_{n}
\end{pmatrix}
= A^{n}
\begin{pmatrix}
f_{1} \\
f_{0}
\end{pmatrix}
$$

Chaque multiplication de A avec elle-même nécessite:
- 4 additions
- 8 multiplications (1 multiplication $\approx$ 256 additions)

Le calcul de `fibo(n)` peut être réalisé avec :
- $4\times n$ additions
- $8\times n$ multiplications ($\approx 2048 \times n$ additions)

Temps de calcul de `fibo(100)` réduit à **200 000** opérations

In [3]:
def mult_mat(A, B):
    dima = size(A)
    dimb = size(B)
    m = dima[0]
    n = dima[1]
    p = dimb[1]
    # A doit être du dimension m*n et B de dimension n*p
    if (n != dimb[0]):
        return
    C = matrix(m, p)
    # Calcul du produit matriciel
    for i in range(0, m, 1):
        for j in range(0, p, 1):
            C[i, j] = 0
            for k in range(0, n, 1):
                C[i, j] = C[i, j] + A[i, k] * B[k, j] 
    return C

## Implémentation avec exponentielle

- Multiplication matricielle sur matrice $2 × 2$: $2052 \times n$ opérations élémentaires (additions)

- Exponentiation rapide : $2\log_2 n$ multiplications matricielles (comme réciproque de $2^n$)

On obtient alors un coût total de l’ordre de $4000 \log_2 n$ additions
- `fibo(100)` peut alors être obtenu en $\approx 26576$ opérations

In [4]:
def id(n):
    C = matrix(n, n)
    for i in range(0, n, 1):
        for j in range(0, n, 1):
            if i == j:
                C[i, j] = 1
            else:
                C[i, j] = 0
    return C

# Pour une matrice carrée
def fast_exp(A, k):
    dima = size(A)
    n = dima[0]
    if k == 0:
        return id(n)
    if mod(k, 2) == 0:
        M = fast_exp(A, k / 2)
        return mult_mat(M, M)
    else:
        M = fast_exp(A, k // 2)
        return mult_mat(A, mult_mat(M, M))

## Conclusion

Trois algorithmes qui permettent de résoudre le même problème :
- L'implémentation récursive est **sub-linéaire**
- L'implémentation multiplication matricielle est **linéaire**
- L'implémentation exponentielle rapide est **supra-linéaire**

**Complexité algorithmique** : Quantification de la charge de travail de cet algorithme en fonction de la taille des entrées.

# Définitions

**Complexité temporelle** (complexité en temps) :

Quantification du nombre d’opérations élémentaires à effectuer pour obtenir le résultat souhaité.


**Complexité spatiale** (complexité en mémoire) :

Quantification du nombre de cases mémoire que la machine doit mettre à disposition du programme pour son exécution.

**Au tableau: illustration complexité spatiale**

**Complexité comme fonction**
$$
C: \mathbb{N} \rightarrow \mathbb{N}
$$
qui prend en entrée la **taille** du problème à résoudre et en sortie le nombre d'**opérations** (ou **cases mémoires**) nécessaires.

L’objectif est de caractériser le comportement asymptotique de l’algorithme et non pas précisemment chiffrer le coût.

Sa croissance est-elle linéaire ? Polynomiale ? Exponentielle ?

# Unités

- Pas de règle absolue
- Choix qui soit représentatif du contenu réel des données
- Doit être justifé


Soit $\mathcal{A}$ l'ensemble des algorithmes.

## Taille des données

Soit $f\in\mathcal{A}$ prenant ses entrées dans un ensemble $\mathcal{D}$. La taille des données est définie par :
$$
\lvert.\rvert : \mathcal{D} \rightarrow \mathbb{N}
$$

### Exemples
- Nombre entier (e.g. factorielle, suite de Fibonacci...)
- Réel, partie entière de ce nombre
- Si plusieurs nombres (coefficient binomial, pgcd, exponentiation), il faudra définir un système en sommant les nombres significatifs (bien souvent un seul des nombres passés en entrée est représentatif de la taille du problème à traiter)
- Si liste (e.g. tri de liste, recherche d’éléments...), on prendra généralement le nombre d’éléments de la liste comme unité de taille.
- Si chaînes de caractères (e.g. recherche de sous-chaine, image miroir...) , sa longueur paraît un bon candidat

Le plus important est de se souvenir que la mesure de taille doit être un **entier naturel**, **représentatif** de la difficulté du problème et **calculable** sans ambiguïté.


## Coût d'exécution

Le coût d’exécution est l’unité de la valeur d’arrivée de la complexité.

L’opération choisie doit être celle qui prend le plus de temps et/ou qui est répétée le plus de fois. Là aussi, nous pouvons remarquer qu’en pratique il n’y a guère d’ambiguïté quant au choix de cette unité.

### Exemples
- Factorielle: multiplication
- pgcd de 2 nombres: 1 opération d'arithmétique modulaire et 3 affectations. Le choix est laissé libre ici, car de toute façon le résultat n'en sera affecté que d'une constante multiplicative.
- Exponentiation rapide: multiplication
- Tri d'une liste: nombre de comparaisons ou affectations


## Conclusion
- Justifier le choix des unités
- Comparaison de deux algorithmes résolvant le même problème: les unités doivent être les mêmes !!!


# Différentes complexités

Rien ne permet d’affirmer que 2 données différentes de même taille n entraînent nécessairement le même nombre d’opérations.

**Complexité minimal** (complexité meilleur cas) :
$$
C_{\min}(n) = \min \{C(d) \mid d \in \mathcal{D}_n\}
$$
- Peu utilisée en pratique
- Sauf sur des cas où les données en entrée sont garanties d'atteindre cette complexité minimale (e.g. donnée triée)

**Complexité maximal** (complexité pire cas) :
$$
C_{\max}(n) = \max \{C(d) \mid d \in \mathcal{D}_n\}
$$
- Elle fournit en quelques sortes une borne inférieure sur les performances de l’algorithme
- Permet de fournir une borne supérieure sur l’espace mémoire à allouer au programme avant de le lancer

**Complexité moyenne** :
$$
\begin{align}
C_{moy}(n) &= \mathbb{E}\left[C(d) \mid d \in \mathcal{D}_n\right]\\
&= \sum^{|\mathcal{D}_n|}_{i=1} C(d_i)\pi(d_i)
\end{align}
$$
où :
- $|D_n|$ le nombre de données de taille $n$ contenues dans $D$.
- $\pi(d_i)$ la probabilité d'occurence de $d_i\in\mathcal{D}_n$

Si distribution uniforme sur $\mathcal{D}_n$ :
$$
\begin{align}
C_{moy}(n) = \frac{1}{|\mathcal{D}_n|}
\sum^{|\mathcal{D}_n|}_{i=1} C(d_i)
\end{align}
$$

### Exemple : Tri par sélection

In [5]:
def selection_sort(a):
    n = size(a)
    for i in range(0, n, 1):
        min = a[i]
        idx = i
        # Recherche de l'indice avec valeur minimale sur le reste du
        # tableau
        for j in range(i + 1, n, 1):
            if (a[j] < min):
                min = a[j]
                idx = j
        # Echange entre l'indice courant et indice valeur minimale
        tmp = a[idx]
        a[idx] = a[i]
        a[i] = tmp

- **Unité de mesure**: taille de la liste `a` que l'on note $n$
- **Opération élémentaire**: comparaison (e.g. `a[j] < min`)

L’algorithme possède deux boucles imbriquées sans débranchement, ni modification des
valeurs de pas ou de borne. Chaque passage dans la boucle contient une comparaison. La
complexité de l’algorithme est donc égal au nombre de passages dans la boucle, soit :
$$
C_{\min}(n) = C_{\moy}(n) = C_{\max}(n) = \frac{n(n-1)}{2}
$$

- **Unité de mesure**: taille de la liste `a` que l'on note $n$
- **Opération élémentaire**: nombre d'échanges à effectuer (on bouge deux éléments distincts de place)

$$
\begin{align}
C_\min(n) &= 0 \\
C_\max(n) &= n &\text{cas où liste totalement inversée} \\
C_{\moy}(n) &= \frac{1}{n!}\sum^{n!}_{1} C(d_i) & \text{car } n! \text{ permutations possibles d'une liste}
\end{align}
$$

- Calculons $C(d_i)$ pour $d_i \in \mathcal{D}_n$ :
  
  Nombre d'échanges à réaliser à l'itération $i$ : $\frac{n-i}{n-i+1}$
  
  $$
  \begin{align}
  C(d_i) &= \sum^{n}_{i=1} \frac{n-i}{n-i+1} \\
  &= \sum^{n}_{i=1} \frac{k-1}{k} \\
  &= n - \sum^{n}_{i=1} \frac{1}{k} \\
  &\sim n - \log n
  \end{align}
  $$

# Complexité multivariée

Attention: Il existe quelques problèmes où les variables sont indépendantes, on est alors obligé de faire du multivarié !

Algorithme de Dijkstra, qui calcule le plus court chemin entre deux points $n$ et $m$ d'un graphe. Complexité pire des cas égale à $C(n, m) = m + n \log n$.

# Notations de Landau

En pratique nous nous intéressons uniquement au comportement de la complexité lorsque
le nombre n tend vers l’infini, l’objectif in fine étant de répartir les algorithmes en différents
groupes de complexité afin de les caractériser et de les comparer entre eux (ceci impliquera
de hiérarchiser les classes d’une manière ou d’une autre).

La complexité étant une fonction à valeurs entières positives, il a alors été nécessaire
d’introduire des classes d’équivalence entre fonctions à valeurs positives


La réponse au problème a été apportée par le mathématicien allemand Edmund Landau (fin XIXeme-début XXeme) qui contribuera à la diffusion des notations qui portent son nom.

# Notations de Landau

En pratique nous nous intéressons uniquement au comportement de la complexité lorsque le nombre n tend vers l’infini, l’objectif in fine étant de répartir les algorithmes en différents groupes de complexité afin de les caractériser et de les comparer entre eux (ceci impliquera
de hiérarchiser les classes d’une manière ou d’une autre).

La complexité étant une fonction à valeurs entières positives, il a alors été nécessaire d’introduire des classes d’équivalence entre fonctions à valeurs positives

La réponse au problème a été apportée par le mathématicien allemand Edmund Landau (fin XIXeme-début XXeme) qui contribuera à la diffusion des notations qui portent son nom.

En analyse de la complexité algorithmique, on va utiliser 3
notations.

### Définition : notation grand $\mathcal{O}$
Soient f et g deux fonctions de
$\mathbb{N} \rightarrow \mathbb{R}^{+∗}$.

On dit que f est **dominée** par g, et on écrit $f = \mathcal{O}(g)$ si et seulement si :
$$
\exists K \in \mathbb{R}^{+∗} : n_0 \in \mathbb{N}
\mid n > n0 \implies f(n) \leq Kg(n)
$$

### Définition : notation grand Θ
Soient f et g deux fonctions de
$\mathbb{N} \rightarrow \mathbb{R}^{+∗}$.

On dit que f est **dominée et soumise** à g, et on écrit f = Θ(g) si et seulement si :
$$
f = \mathcal{O}(g) \land g = \mathcal{O}(f)
$$

### Définition: Équivalence
Soient f et g deux fonctions de
$\mathbb{N} \rightarrow \mathbb{R}^{+∗}$.

On dit que f est **équivalente** à g et on note $f \sim g$ si et seulement si :
$$
\forall \varepsilon > 0
\exists n_0 \in \mathbb{N}
\mid n > n0 \implies |f (n) − g(n)| \leq \varepsilon g(n)
$$

# Classes de complexité

**Remarque 1** : d’une manière générale, la classe hyperexponentielle regroupe l’ensemble des algorithmes dont la complexité C est telle que le rapport $C(n)/a^{n}$ tende vers +∞ pour toute valeur de a ∈ R+∗

**Remarque 2**: la base du logarithme n’est pas importante.

Voir Table 3.2 et 3.3 du cours (page 104 du polycopié)

**Remarque 3**: notons qu’un algorithme quasi-linéaire appartient stricto sensus à la classe supra-linéaire. En pratique, pas de différence significatives entre un algorithme en $n \log n$ et $n$

**Remarque 4**: en théorie de la complexité pour les problèmes de décision, la distinction s’opère principalement entre les algorithmes au plus polynomiaux d’une part, et les algorithmes exponentiels (et hyperexponentiels) d’autre part. C’est en particulier l’objet du
fameux problème P = NP (voir Garey et al, 1991 par exemple).

**Remarque 5**: la comparaison n’est valide que lorsque $n$ est suffisamment grand. Pour des données de petites tailles, il arrive bien souvent qu’un algorithme en temps polynomial par exemple, soit plus rapide qu’un algorithme quasi-linéaire (en particulier si le second
nécessite quantité de pré-traitements sophistiqués, effectués en temps constant certes, mais qui ne sont rentabilisés que pour des données volumineuses). On parle de **hidden constant factor** (e.g cas GTAV).

**Remarque 6**: on se gardera également de faire des comparaisons générales entre algorithmes conçus pour des tâches différentes.

**Remarque 7**: Lorsque l'entrée de l’algorithme est un nombre entier, il est parfois d’usage (en particulier en théorie de la complexité) de prendre comme unité de taille la longueur du nombre dans sa décomposition en bits.

Attention donc à bien considérer l’unité de mesure de la taille utilisée avant d’interpréter la complexité indiquée pour un algorithme

### Exemple : évaluation polynômiale

In [6]:
def eval(p, x):
    n = size(p)
    s = 0
    for i in range(0, n, 1):
        s = s + p[i] * x ** i
    return s

P = [1, 2, 3]
x = 2
eval(P, x)

17

En utilisant seulement des multiplications :
- **Unités**:
  + **entrées**: taille du polynôme
  + **opération élémentaires**: multiplication
- **Complexité**
  + $i$ multiplication à chaque passage dans la boucle
  + $n$ itérations de boucle
  + $C(n) = n(n+1)/2 = O(n^2)$
  
En utilisant l'exponentiation rapide :
  + puissance en $2\log_2 n$ opérations
  + $C(n) = \dots$
  
Par la formule de Stirling, quasi-linéaire:
$$
C_{\max} = O(n \log n)
$$

In [7]:
# En utilisant la méthode de Ruffini-Horner
# Par exemple: a3*x^3+a2*x^2+a1*x+a0 = ((a3*x+a2)x+a1)x+a0

def eval(p, x):
    n = size(p)
    s = p[n - 1]
    for i in range(n - 2, -1, -1):
        s = s * x
        s = s + p[i]
    return s

$$
C_{\min} = C_{\moy} = C_{\max}(n) = O(n)
$$

# Théorème d'encadrement

### Théorème : encadrement de la complexité moyenne

Soit f une fonction de N → R+∗ :
Cmin ∈ Ω(f ) et Cmax ∈ O(f ) ⇒ Cmoy ∈ Θ(f )

# Méthodologie

Stratégie :
- Choix d’une **unité de taille** pour caractériser la donnée en entrée
- Choix d’une **unité de coût de complexité** (i.e. choix d’une opération élémentaire)
- Recherche des cas dont ont peu démontrer qu’ils sont (respectivement) le plus favorable et le plus défavorable en terme de nombre d’opérations. Il suffit alors d’appliquer l’algorithme sur ces cas et de dénombrer les opérations effectuées en tenant compte des mises-à-jour de compteur de boucle et des débranchements. Si possible, on souhaite garder une expression analytique exacte du nombre d’opérations ($C_\min$ et $C_\max$).
- $C_\min = C_\max$ ?
  + Si oui, par le théorème d'encadrement $C_{\moy} = C_\max = C_\min$
  + Sinon, on se munit d'une distribution de probabilité sur l'ensemble (fini !) des données acceptées en entrées par l'algorithme. On recherche une expression analytique de la complexité de chaque donnée et on calcule la moyenne pondérée (par les probabilités d’apparition) de ces valeurs de complexités. Là aussi, on cherchera à conserver une expression analytique exacte le plus longtemps possible.
- Enfin, on cherche l’ordre (si possible avec un grand Θ) des complexités trouvées et on détermine les classes de complexité correspondantes (logarithmique, linéaire, polynomiale...).

### Exemple: recherche dans un tableau

Retourne l'indice de l'élément trouvé ou $-1$ si l'élément n'est pas présent

In [8]:
def find(t, e):
    n = size(t)
    for i in range(0, n, 1):
        if t[i] == e:
            return i
    return -1

# Cas des fonctions récursives

## Récurrence linéaire simple

$$
T(n) = aT(n-1) + f(n)
$$

**Cas** $a=1$
$$
T(n) = T(1) + \sum^{n-1}_{k=1} f(k)
$$

**Cas** $a\leq 2$
$$
\begin{align}
u_n &= \frac{T(n)}{a^{n}} \\
u_n &= u_{n-1} + \frac{f(n)}{a^{n}} \\
&= u_1 + \sum^{n-1}_{k=1} \frac{f(k)}{a^k}
\end{align}
$$

## Récurrence linéaire multiple
Voir au tableau.


## Diviser pour régner

$$
T(n) = aT(\frac{n}{b}) + f(n)
$$

### Master theorem
Etant donnés deux entiers a, b ∈ N∗ ainsi qu’une fonction T : N → R+ , on appelle équation
de partition la relation de récurrence :
T (n) = a T
n
b
+ f (n)
où la quantité n/b est à comprendre au sens de la division entière, et où f est une fonction
croissante de N → R+ .

1. Si pour un réel ε > 0 f (n) = O(nlogb a−ε )
2. Si f (n) = Θ(nlogb a )
⇒
T (n) = Θ(nlogb a log n)
⇒
3. Si pour un réel ε > 0 f (n) = Ω(nlogb a+ε )
T (n) = Θ(nlogb a )
⇒
T (n) = Θ(f (n))
Dans le cas 3, on devra aussi vérifier la condition de régularité, c’est-à-dire que :
n
∀n∈N

### Stratégie générale

1. on découpe le problème en 2 (ou plus) parties et on suppose connaître la solution sur chacun des sous-problèmes.
2. on étudie si la solution du problème principal peut être exprimée comme une combinaison de la solution sur chaque sous partie, plus éventuellement d’un terme d’ajustement faisant intervenir les deux parties (en général, si cet ajustement est trop coûteux en temps de calcul, c’est que le découpage choisi à l’étape 1 n’est pas judicieux ou alors que l’approche diviser pour régner n’est pas adaptée au problème considéré).
3. on étudie le cas de base de la récurrence, i.e. on montre que l’on peut donner facilement la solution d’un sous-problème élémentaire
4. on évalue le coût $f(n)$ de décomposition/recomposition à l’étage n.
5. on utilise le Master Theorem (les valeurs de a et b dépendent de l’étape 1), pour vérifier que l’approche possède une complexité plus faible que la solution itérative.